In [1]:
import xarray as xr
import rioxarray
import numpy as np

print(np.__version__)
print(xr.__version__)
print(rioxarray.__version__)

1.23.5
2023.6.0
0.15.0


In [2]:
# Step 1: Load NetCDF file
file_path = r'C:\Users\alexl\Downloads\climatology-hd30-annual-mean_cmip6-x0.25_access-cm2-r1i1p1f1-ssp126_climatology_mean_2020-2039.nc'
nc = xr.open_dataset(file_path)

In [3]:
nc

<xarray.Dataset>
Dimensions:                       (time: 1, lat: 721, lon: 1440, bnds: 2)
Coordinates:
  * time                          (time) datetime64[ns] 2020-07-01
  * lat                           (lat) float32 -90.0 -89.75 ... 89.75 90.0
  * lon                           (lon) float32 -180.0 -179.8 ... 179.5 179.8
  * bnds                          (bnds) int32 0 1
Data variables:
    climatology-hd30-annual-mean  (time, lat, lon) timedelta64[ns] ...
    lon_bnds                      (lon, bnds) float64 ...
    lat_bnds                      (lat, bnds) float64 ...
Attributes: (12/15)
    wb_truncation_label:   2
    wb_grid_label:         x0.25
    wb_period_label:       2020-2039
    wb_percentile_label:   mean
    wb_type_label:         climatology
    wb_model_label:        access-cm2
    ...                    ...
    wb_aggregation_label:  annual
    wb_variable_label:     hd30
    wb_product_label:      climatology
    wb_collection_label:   cmip6-x0.25
    wb:                     \nwb_data_citation = Please acknowledge data sour...
    wb_CMIP6:               \nwb_creation_date = Fri Mar 17 08:02:44 PDT 2023...

In [4]:
# Step 2: Display available dimensions and variables
print("Dimensions:\n", nc.dims)
print("\nVariables:\n", nc.data_vars)

Dimensions:
 Frozen({'time': 1, 'lat': 721, 'lon': 1440, 'bnds': 2})

Variables:
 Data variables:
    climatology-spei12-annual-mean  (time, lat, lon) float32 ...
    lon_bnds                        (lon, bnds) float64 ...
    lat_bnds                        (lat, bnds) float64 ...


In [5]:
# Step 3: Select a variable and specific dimensions
variable_name = 'climatology-spei12-annual-mean'  # Replace with your variable name
selected_var = nc[variable_name]

In [6]:
# Optional: If the variable has multiple dimensions (e.g., time), select a specific slice
if 'time' in selected_var.dims:
    selected_var = selected_var.isel(time=0)  # Select the first time slice

In [7]:
# Ensure the DataArray has the necessary spatial metadata
if not selected_var.rio.crs:
    selected_var = selected_var.rio.write_crs("EPSG:4326", inplace=True)

In [8]:
type(selected_var)

xarray.core.dataarray.DataArray

In [9]:
print(selected_var.rio.crs)  # Should not be None
print(selected_var.rio.bounds())  # Should return valid bounds
print(selected_var)

EPSG:4326
(-180.125, -90.125, 179.875, 90.125)
<xarray.DataArray 'climatology-spei12-annual-mean' (lat: 721, lon: 1440)>
[1038240 values with dtype=float32]
Coordinates:
    time         datetime64[ns] 2020-07-01
  * lat          (lat) float32 -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * lon          (lon) float32 -180.0 -179.8 -179.5 -179.2 ... 179.2 179.5 179.8
    spatial_ref  int32 0
Attributes:
    climatoloyg_calculation:  avg
    annual_calculation:       avg
    units:                    index
    _FillValue_original:      1.0000000200408773e+20
    long_name:                Standardized Precipitation Evapotranspiration I...
    valid_min:                nan
    valid_max:                nan


In [11]:
# Export to raster
selected_var.rio.to_raster(r'C:\Users\alexl\Dropbox\GitHub\AgCAP\GIS_data\spei12\spei12.tif')